### Notes

The models you can run here will be limited by what's implemented by the litellm library, because that's what managing the API calls.

You can see the models here: https://docs.litellm.ai/docs/



In [1]:
from functions import *


C:\Users\abiga\OneDrive\Documents\PythonScripts\llm_eval\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameters

In [2]:
models_dict = {
    #'claude-2.1':  "ANTHROPIC", 
    'gpt-3.5-turbo-0301': "OPENAI"
               } 
csv_file_path = '../data/prompt_target_answer_pairs.csv'
similarity_model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
num_runs = 1
temperature = .9
num_perturbations= 2
is_file_path = True
llm_evaluation_model = ['gpt-4', "OPENAI"]
instructions = "Please answer thoroughly: "
perturbation_model = ['gpt-4', "OPENAI"] # I recommend using a good model for perturbations otherwise it may generate the wrong number

pipeline = LLMAnalysisPipeline(
    input_data=csv_file_path, 
    models_dict=models_dict, 
    perturbation_model=perturbation_model, 
    llm_evaluation_model=llm_evaluation_model,
    temperature = temperature,
    num_runs= num_runs,
    is_file_path = is_file_path,
    similarity_model_name = similarity_model_name,
    num_perturbations = num_perturbations,
    instructions = instructions

)

# Run the pipeline
df_responses = pipeline.run_pipeline()



2023-12-14 10:04:34,440 - INFO - Reading prompts from CSV file.
2023-12-14 10:04:42,937 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'What is the powerhouse of the cell and how does it work?': ['What is the powerhouse of the cell and how does it work?',
  "- Can you explain the function and operation mechanism of the cell's powerhouse?",
  "- Could you describe the cell's powerhouse and its working principles?",
  '- What is known as the powerhouse of the cellular structure and how does it function?',
  '- What is the role of the powerhouse within a cell and how does it fulfill its function?']}

In [7]:
df_responses.groupby(['model', 'original_prompt', 'actual_prompt']).count()

response  \
model              original_prompt                                    actual_prompt                                                  
gpt-3.5-turbo-0301 How do astroflora perform photosynthesis in low... How do astroflora perform photosynthesis in low...         2   
                                                                      In what way do astroflora carry out photosynthe...         2   
                                                                      What is the method by which astroflora complete...         2   
                   What do genes consist of and how do they work?     Can you explain the composition of genes and de...         2   
                                                                      What are the components of genes and how do the...         2   
                                                                      What do genes consist of and how do they work?             2   
                   What is the basic unit of life and what does th... Can you define the primary unit of life and ela...         2   
                                                                      What does it mean when we refer to the fundamen...         2   
                                                                      What is the basic unit of life and what does th...         2   
                   What is the powerhouse of the cell and how does... Could you describe the role and working process...         2   
                                                                      What is the powerhouse of the cell and how does...         2   
                                                                      What is the role of the powerhouse in a cell an...         2   
                   What particle stabilizes the membrane of hover ... What particle stabilizes the membrane of hover ...         2   
                   Which organelle contains the cell's futurogenic... Can you identify the organelle that holds the m...         2   
                                                                      Which cell component houses the future genetic ...         2   
                                                                      Which organelle contains the cell's futurogenic...         2   

                                                                                                                          temperature  \
model              original_prompt                                    actual_prompt                                                     
gpt-3.5-turbo-0301 How do astroflora perform photosynthesis in low... How do astroflora perform photosynthesis in low...            2   
                                                                      In what way do astroflora carry out photosynthe...            2   
                                                                      What is the method by which astroflora complete...            2   
                   What do genes consist of and how do they work?     Can you explain the composition of genes and de...            2   
                                                                      What are the components of genes and how do the...            2   
                                                                      What do genes consist of and how do they work?                2   
                   What is the basic unit of life and what does th... Can you define the primary unit of life and ela...            2   
                                                                      What does it mean when we refer to the fundamen...            2   
                                                                      What is the basic unit of life and what does th...            2   
                   What is the powerhouse of the cell and how does... Could you describe the role and working process...            2   
                                                                 

In [9]:
aggregate_best_scores(df_responses, "similarity_scores")

KeyError: 'Column not found: similarity_scores'

In [ ]:
aggregate_best_scores(df_responses, "similarity_scores")

In [ ]:
aggregate_best_scores(df_responses, 'keyword_scores')

In [ ]:
aggregate_best_scores(df_responses, 'llm_ratings')